In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

In [2]:
# Carregar a tabela de modelagem (resultado do notebook 06)
path_base = '../data/processed/modeling_table.parquet'
df_base = pd.read_parquet(path_base)
print(f"Tabela base de modelagem carregada com {df_base.shape[0]} registros.")

# --- Função para carregar múltiplos arquivos (mensais ou anuais) ---
def carregar_dados_multiplos(caminho_padrao, separador=';', decimal=','):
    """Função para encontrar, ler e concatenar múltiplos arquivos CSV."""
    lista_arquivos = glob.glob(caminho_padrao)
    if not lista_arquivos:
        raise FileNotFoundError(f"Nenhum arquivo encontrado para o padrão: {caminho_padrao}")
    lista_dfs = []
    for arquivo in lista_arquivos:
        print(f"Lendo o arquivo: {os.path.basename(arquivo)}")
        df_parte = pd.read_csv(arquivo, sep=separador, decimal=decimal)
        lista_dfs.append(df_parte)
    return pd.concat(lista_dfs, ignore_index=True)

try:
    print("\n--- Carregando dados de CMO Semanal ---")
    padrao_cmo = '../data/raw/CMO_SEMANAL_*.csv'
    df_cmo_raw = carregar_dados_multiplos(padrao_cmo)

    print("\n--- Carregando dados de Disponibilidade de Usinas ---")
    padrao_disp = '../data/raw/DISPONIBILIDADE_USINA_*.csv'
    df_disp_raw = carregar_dados_multiplos(padrao_disp)

except Exception as e:
    print(f"ERRO CRÍTICO: Falha ao ler os arquivos CSV. Verifique os nomes e caminhos. Erro: {e}")
    raise

Tabela base de modelagem carregada com 2781 registros.

--- Carregando dados de CMO Semanal ---
Lendo o arquivo: CMO_SEMANAL_2018.csv
Lendo o arquivo: CMO_SEMANAL_2023.csv
Lendo o arquivo: CMO_SEMANAL_2022.csv
Lendo o arquivo: CMO_SEMANAL_2024.csv
Lendo o arquivo: CMO_SEMANAL_2021.csv
Lendo o arquivo: CMO_SEMANAL_2020.csv
Lendo o arquivo: CMO_SEMANAL_2019.csv
Lendo o arquivo: CMO_SEMANAL_2025.csv

--- Carregando dados de Disponibilidade de Usinas ---
Lendo o arquivo: DISPONIBILIDADE_USINA_2021_03.csv
Lendo o arquivo: DISPONIBILIDADE_USINA_2019_10.csv
Lendo o arquivo: DISPONIBILIDADE_USINA_2019_12.csv
Lendo o arquivo: DISPONIBILIDADE_USINA_2019_11.csv
Lendo o arquivo: DISPONIBILIDADE_USINA_2024_02.csv
Lendo o arquivo: DISPONIBILIDADE_USINA_2020_05.csv
Lendo o arquivo: DISPONIBILIDADE_USINA_2021_02.csv
Lendo o arquivo: DISPONIBILIDADE_USINA_2024_08.csv
Lendo o arquivo: DISPONIBILIDADE_USINA_2022_07.csv
Lendo o arquivo: DISPONIBILIDADE_USINA_2021_01.csv
Lendo o arquivo: DISPONIBILIDADE_US

In [3]:
# --- Célula 3: Processamento - CMO Semanal (VERSÃO FINAL CORRIGIDA) ---

# Diagnóstico para confirmar o nome do subsistema
print("--- Diagnóstico: Valores únicos na coluna 'nom_subsistema' do CMO ---")
print(df_cmo_raw['nom_subsistema'].unique())
print("---------------------------------------------------------------------")

coluna_data_cmo = 'din_instante'
coluna_valor_cmo = 'val_cmomediasemanal'

# Filtra para o subsistema 'SUDESTE'
df_cmo_seco = df_cmo_raw[df_cmo_raw['nom_subsistema'] == 'SUDESTE'].copy()

# Força a conversão numérica para segurança
df_cmo_seco[coluna_valor_cmo] = pd.to_numeric(df_cmo_seco[coluna_valor_cmo], errors='coerce')

df_cmo_seco['timestamp'] = pd.to_datetime(df_cmo_seco[coluna_data_cmo])
df_cmo_seco.set_index('timestamp', inplace=True)

# --- CORREÇÃO: Ordenamos o índice cronologicamente ---
# Isso é necessário para que o 'reindex' com 'ffill' funcione.
df_cmo_seco.sort_index(inplace=True)
# ---------------------------------------------------

# Seleciona apenas a coluna de valor que nos interessa
df_cmo_seco = df_cmo_seco[[coluna_valor_cmo]]

# Cria um dataframe diário e preenche com os valores semanais de CMO
cmo_diario = df_cmo_seco.reindex(df_base.index, method='ffill')

# Atribuímos o resultado de volta para preencher os NaNs do início e evitar o warning
cmo_diario = cmo_diario.bfill()

cmo_diario.rename(columns={coluna_valor_cmo: 'cmo_semanal_seco'}, inplace=True)

print("\nFeature de CMO diário criada com sucesso.")
display(cmo_diario.head())

--- Diagnóstico: Valores únicos na coluna 'nom_subsistema' do CMO ---
['NORTE' 'NORDESTE' 'SUL' 'SUDESTE']
---------------------------------------------------------------------

Feature de CMO diário criada com sucesso.


,cmo_semanal_seco
timestamp,
2018-01-31,172.05
2018-02-01,172.05
2018-02-02,157.33
2018-02-03,157.33
2018-02-04,157.33


In [4]:
# --- Célula 4: Processamento - Disponibilidade de Usinas (VERSÃO CORRIGIDA) ---

# Inspecionar colunas
print("Colunas de Disponibilidade:", df_disp_raw.columns.tolist())

# --- CORREÇÃO: Usando os nomes de colunas corretos do arquivo ---
coluna_data_disp = 'din_instante'
coluna_estado_disp = 'id_estado'
coluna_valor_disp = 'val_dispoperacional' # O nome correto é este
# -------------------------------------------------------------

df_disp = df_disp_raw[[coluna_data_disp, coluna_estado_disp, coluna_valor_disp]].copy()

# Força a conversão numérica
df_disp[coluna_valor_disp] = pd.to_numeric(df_disp[coluna_valor_disp], errors='coerce')
df_disp.fillna(0, inplace=True)

# Converte data, filtra para Goiás e agrega por dia
df_disp['timestamp'] = pd.to_datetime(df_disp[coluna_data_disp])
df_disp_go = df_disp[df_disp[coluna_estado_disp] == 'GO'].copy()
df_disp_go.set_index('timestamp', inplace=True)
disponibilidade_diaria = df_disp_go[[coluna_valor_disp]].resample('D').sum()
disponibilidade_diaria.rename(columns={coluna_valor_disp: 'disponibilidade_total_diaria_go'}, inplace=True)

print("\nFeature de Disponibilidade diária criada com sucesso.")
display(disponibilidade_diaria.head())

Colunas de Disponibilidade: ['id_subsistema', 'nom_subsistema', 'id_estado', 'nom_estado', 'nom_usina', 'id_tipousina', 'nom_tipocombustivel', 'id_ons', 'ceg', 'din_instante', 'val_potenciainstalada', 'val_dispoperacional', 'val_dispsincronizada']

Feature de Disponibilidade diária criada com sucesso.


,disponibilidade_total_diaria_go
timestamp,
2018-01-01,72260.421500
2018-01-02,72263.812667
2018-01-03,72261.620667
2018-01-04,72414.748167
2018-01-05,73373.488000


In [5]:
# Começa com a tabela de modelagem do notebook anterior
df_final = df_base.copy()

# Adiciona as novas features
df_final = df_final.join(cmo_diario)
df_final = df_final.join(disponibilidade_diaria)

# Preenche possíveis nulos que sobraram (ex: início do período para o CMO)
df_final.fillna(method='bfill', inplace=True) # Backward fill
df_final.fillna(0, inplace=True) # Garante que não sobra nenhum

print("Tabela final enriquecida:")
df_final.info()

# Salvar a nova tabela, pronta para uma nova rodada de modelagem
output_path = '../data/processed/modeling_table_v2.parquet'
df_final.to_parquet(output_path)
print(f"\nNova tabela para modelagem salva com sucesso em: {output_path}")

Tabela final enriquecida:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2781 entries, 2018-01-31 to 2025-09-11
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   deficit_diario_mwh               2781 non-null   float64
 1   nivel_risco                      2781 non-null   object 
 2   programada                       2781 non-null   float64
 3   verificada                       2781 non-null   float64
 4   diferenca_verif_prog             2781 non-null   float64
 5   geracao_total_diaria_go          2781 non-null   float64
 6   geracao_fotovoltaica_diaria      2781 non-null   float64
 7   geracao_hidroelétrica_diaria     2781 non-null   float64
 8   geracao_térmica_diaria           2781 non-null   float64
 9   total_mwh_restrito_go            2781 non-null   float64
 10  saldo_intercambio_seco           2781 non-null   float64
 11  ear_percentual_seco              2781 

/tmp/ipykernel_851414/931758909.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_final.fillna(method='bfill', inplace=True) # Backward fill
